In [102]:
%pip install scipy statsmodels tqdm
import numpy as np
import scipy.stats as stats
from tqdm import tqdm as tqdm
import pandas as pd
import random
import pandas as pd
from scipy.optimize import bisect
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [103]:
df = pd.read_csv(
    filepath_or_buffer="data2.csv"
)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 799061 entries, 0 to 799060
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   user_id     799061 non-null  object
 1   gmv         799061 non-null  int64 
 2   group_name  799061 non-null  object
dtypes: int64(1), object(2)
memory usage: 18.3+ MB


In [104]:

groupped = df.groupby(by=["user_id", "group_name"]).agg("sum").reset_index()
control = groupped[groupped['group_name'] == 'control']['gmv']
test = groupped[groupped['group_name'] == 'test']['gmv']
t_stat, p_value = stats.ttest_ind(test, control, equal_var=False)
rounded_t = round(t_stat, 3)
rounded_p = round(p_value, 3)


In [105]:
rounded_t

np.float64(2.36)

In [106]:
rounded_p

np.float64(0.018)

In [107]:

df = pd.read_csv("data2.csv")


user_gmv = df.groupby(['user_id', 'group_name'])['gmv'].sum().reset_index()


control = user_gmv[user_gmv['group_name'] == 'control']['gmv']
test = user_gmv[user_gmv['group_name'] == 'test']['gmv']


t_stat2, p_value2 = stats.ttest_ind(test, control, equal_var=False)



In [108]:
t_stat2

np.float64(2.3604123426626256)

In [109]:
p_value2

np.float64(0.018256922180307746)

In [110]:
import pandas as pd
from scipy import stats

# Загрузка данных
df = pd.read_csv("data2.csv")

# Добавляем признак транзакции
df['transactions'] = 1

# Агрегируем до пользователя: сумма GMV и кол-во транзакций
user_level = df.groupby(['user_id', 'group_name']).agg({
    'gmv': 'sum',
    'transactions': 'sum'
}).reset_index()

# Считаем средний чек на пользователя
user_level['mean_check'] = user_level['gmv'] / user_level['transactions']

# Разбиваем на группы
control = user_level[user_level['group_name'] == 'control']['mean_check']
test = user_level[user_level['group_name'] == 'test']['mean_check']

# Считаем t-статистику и p-value
t_stat, p_val = stats.ttest_ind(test, control, equal_var=False)

# Вывод
print(f"{t_stat:.3f} {p_val:.3f}")


2.971 0.003


In [113]:
import pandas as pd
import numpy as np
from scipy import stats

# Загрузка экспериментальных данных
experiment_df = pd.read_csv("data2.csv")

# Расчет агрегированных метрик по юзерам
user_metrics = experiment_df.groupby(['user_id', 'group_name']).agg(
    sum_transactions=('gmv', 'sum'),
    count_transactions=('gmv', 'count')
).reset_index()

# Вычисление среднего чека
user_metrics['avg_transaction'] = user_metrics['sum_transactions'] / user_metrics['count_transactions']

# Разделение на экспериментальные группы
control_group_metrics = user_metrics[user_metrics['group_name'] == 'control']
test_group_metrics = user_metrics[user_metrics['group_name'] == 'test']

# Извлечение значений среднего чека
control_avg_values = control_group_metrics['avg_transaction'].values
test_avg_values = test_group_metrics['avg_transaction'].values

# Параметры выборок
num_control = len(control_avg_values)
num_test = len(test_avg_values)
degrees_of_freedom = num_control + num_test - 2

# Статистические характеристики
mean_control = np.mean(control_avg_values)
mean_test = np.mean(test_avg_values)

# Несмещенные оценки дисперсии
variance_control = np.var(control_avg_values, ddof=1)
variance_test = np.var(test_avg_values, ddof=1)

# Объединенная дисперсия (дельта-метод)
combined_variance = ((num_control-1)*variance_control + (num_test-1)*variance_test) / degrees_of_freedom

# Стандартная ошибка разности
std_error_diff = np.sqrt(combined_variance * (1/num_control + 1/num_test))

# Расчет t-статистики
t_value_result = (mean_test - mean_control) / std_error_diff

# Расчет p-value (двусторонний тест)
p_value_result = 2 * (1 - stats.t.cdf(abs(t_value_result), degrees_of_freedom))

# Форматирование результатов
formatted_t = round(t_value_result, 3)
formatted_p = round(p_value_result, 3)

# Вывод финальных результатов
print(f"{formatted_t:.3f} {formatted_p:.3f}")

2.983 0.003


In [114]:
import pandas as pd
import numpy as np
from scipy import stats

# Загрузка данных
df = pd.read_csv("data2.csv")

# 1. Агрегация до уровня пользователя
user_stats = df.groupby(['user_id', 'group_name']).agg(
    total_gmv=('gmv', 'sum'),
    tx_count=('gmv', 'count')
).reset_index()

# 2. Расчет среднего чека на пользователя
user_stats['avg_check'] = user_stats['total_gmv'] / user_stats['tx_count']

# 3. Разделение на группы
control = user_stats[user_stats['group_name'] == 'control']['avg_check']
test = user_stats[user_stats['group_name'] == 'test']['avg_check']

# Вариант 1: Встроенный t-test с поправкой Уэлча (equal_var=False)
t_welch, p_welch = stats.ttest_ind(test, control, equal_var=False)
print(f"С поправкой Уэлча: {t_welch:.3f} {p_welch:.3f}")

# Вариант 2: Ручной расчет с упрощенными степенями свободы (n+m-2)
n, m = len(control), len(test)
df_simple = n + m - 2

# Разность средних
mean_diff = test.mean() - control.mean()

# Объединенная дисперсия
pooled_var = ((n-1)*control.var(ddof=1) + (m-1)*test.var(ddof=1)) / df_simple

# Стандартная ошибка
se = np.sqrt(pooled_var * (1/n + 1/m))

# t-статистика
t_simple = mean_diff / se

# p-value
p_simple = 2 * (1 - stats.t.cdf(abs(t_simple), df_simple))
print(f"С упрощенными df: {t_simple:.3f} {p_simple:.3f}")

# Итоговый ответ (оба варианта верны, можно выбрать любой)
final_t = round(t_welch, 3)  # или round(t_simple, 3)
final_p = round(p_welch, 3)  # или round(p_simple, 3)
print(f"Финальный ответ: {final_t:.3f} {final_p:.3f}")

С поправкой Уэлча: 2.971 0.003
С упрощенными df: 2.983 0.003
Финальный ответ: 2.971 0.003


In [115]:
import pandas as pd
import numpy as np
from scipy import stats

# Загрузка данных
df = pd.read_csv("data2.csv")

# Агрегация по пользователям
user_data = df.groupby(['user_id', 'group_name']).agg(
    user_gmv=('gmv', 'sum'),
    user_tx=('gmv', 'count')
).reset_index()

# Разделение на группы
control = user_data[user_data['group_name'] == 'control']
test = user_data[user_data['group_name'] == 'test']

# Функция для дельта-метода
def delta_method(gmv, tx):
    mean_gmv = np.mean(gmv)
    mean_tx = np.mean(tx)
    var_gmv = np.var(gmv, ddof=1)
    var_tx = np.var(tx, ddof=1)
    cov = np.cov(gmv, tx, ddof=1)[0,1]
    
    variance = (var_gmv/mean_tx**2) + (mean_gmv**2*var_tx/mean_tx**4) - (2*mean_gmv*cov/mean_tx**3)
    return variance / len(gmv)  # делим на n для дисперсии среднего

# Применяем к группам
var_control = delta_method(control['user_gmv'], control['user_tx'])
var_test = delta_method(test['user_gmv'], test['user_tx'])

# Средние чеки
theta_control = control['user_gmv'].sum() / control['user_tx'].sum()
theta_test = test['user_gmv'].sum() / test['user_tx'].sum()

# t-статистика
t_value = (theta_test - theta_control) / np.sqrt(var_control + var_test)

# Степени свободы (упрощенная формула)
df = len(control) + len(test) - 2

# p-value
p_value = 2 * (1 - stats.t.cdf(abs(t_value), df))

# Округление
print(f"{t_value:.3f} {p_value:.3f}")

2.344 0.019
